# Testing Retrieval-Augmented Generation (RAG) with Amazon Bedrock

This notebook demonstrates:

- Implementation of RAG techniques using Amazon Bedrock
- Enhancement of LLMs with external knowledge
- Utilization of custom knowledge bases
- Testing of retrieval functionality

Ideal for developers and AI enthusiasts exploring how to augment language models with domain-specific knowledge for improved accuracy and relevance.


⭐ Do you find this project usefull? Please consider giving it a star on GitHub! 

💖 Made with dedication to help fellow developers. Your support means the world! 

In [1]:
import boto3
import pprint

In [2]:
# import kb_id from my private file config.py
from config import *

In [3]:
pp = pprint.PrettyPrinter(indent=2)
region_name = 'us-east-1'
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name = region_name)
bedrock_client = boto3.client('bedrock', region_name=region_name)

<a id="test-a-knowledge-base"></a>
## AWS Bedrock Agent with Knowledge Base Integration

This script demonstrates how to use AWS Bedrock Agent Runtime to perform RAG (Retrieval-Augmented Generation) operations with a custom knowledge base. It includes:

- Setup for Bedrock Agent Runtime client
- A function to query the LLM with knowledge base support
- Example usage with Claude 3 Sonnet model
- Handling of session IDs for follow-up questions
- Citation retrieval from the knowledge base

🚀 Ideal for developers working on AI-powered Q&A systems or chatbots with custom knowledge bases.

In [4]:
response = bedrock_client.list_foundation_models()
for ele in response['modelSummaries']:
    print(ele['modelArn'])

arn:aws:bedrock:us-east-1::foundation-model/twelvelabs.pegasus-1-2-v1:0
arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-opus-4-1-20250805-v1:0
arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large
arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0
arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1
arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v2:0
arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-premier-v1:0:8k
arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-premier-v1:0:20k
arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-premier-v1:0:1000k
arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-premier-v1:0:mm
arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-premier-v1:0
arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-premier-v1:0
arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0:24k
arn:aws:bedrock:us-east-1::foundation-mo

In [5]:
response = bedrock_client.list_inference_profiles()
for ele in response['inferenceProfileSummaries']:
    print(ele['inferenceProfileArn'])

arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.anthropic.claude-3-sonnet-20240229-v1:0
arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.anthropic.claude-3-opus-20240229-v1:0
arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.anthropic.claude-3-haiku-20240307-v1:0
arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.meta.llama3-2-11b-instruct-v1:0
arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.meta.llama3-2-3b-instruct-v1:0
arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.meta.llama3-2-90b-instruct-v1:0
arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.meta.llama3-2-1b-instruct-v1:0
arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.anthropic.claude-3-5-sonnet-20240620-v1:0
arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.anthropic.claude-3-5-haiku-20241022-v1:0
arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.meta.llama3-1-8b-instruct-v1:0
arn:aws:bedrock:us-east-1:609009159737:infe

In [6]:
pp = pprint.PrettyPrinter(indent=2)
region_name = 'us-east-1'
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name = region_name)

In [7]:
model_id = "arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.anthropic.claude-3-5-sonnet-20241022-v2:0"
kb_id = kb_id

In [8]:
def ask_bedrock_llm_with_knowledge_base(query: str, model_arn: str, kb_id: str, sessionId:str = None) -> str:
    response = "None"
    # If this is the first RAG action
    if not sessionId:
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={
                'text': query
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': model_arn
                }
            },
        )
    else:
        # If sessionId is present
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={
                'text': query
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': model_arn
                }
            },
            sessionId = sessionId
        )       

    return response

In [9]:
query = 'what are the headquarters of next tech innovation'


model_arn = model_id
response = ask_bedrock_llm_with_knowledge_base(query, model_arn, kb_id)

for i, ele in enumerate(response['citations']):
    print(f"citation {i}")
    for key, value in ele.items():
        print(f"--- key{key}")
        print(value)

citation 0
--- keygeneratedResponsePart
{'textResponsePart': {'span': {'end': 103, 'start': 0}, 'text': 'NexTech Innovations is headquartered in Boston, Massachusetts at 123 Innovation Drive, Boston, MA 02110.'}}
--- keyretrievedReferences
[{'content': {'text': "Here's a sample company profile text file:\r \r NEXTECH INNOVATIONS, INC.\r Established: 2008\r Headquarters: Boston, Massachusetts\r CEO: Dr. Sarah Chen\r Employees: 2,500+\r \r Company Overview:\r NexTech Innovations is a leading provider of artificial intelligence and robotics solutions, specializing in industrial automation and smart manufacturing technologies. Founded by Dr. Sarah Chen and Dr. Marcus Rodriguez, the company has grown from a small startup to a global technology leader.", 'type': 'TEXT'}, 'location': {'s3Location': {'uri': 's3://virgilio-murillo-bucket/tests/bedrock/knowledge-base-tests/company-card.txt'}, 'type': 'S3'}, 'metadata': {'x-amz-bedrock-kb-source-uri': 's3://virgilio-murillo-bucket/tests/bedrock/k

In [10]:
sessionId = response['sessionId']
print(sessionId)

38b9a71a-33d3-42a8-9a53-87bb0f97f1b7


In [11]:
import time
time.sleep(70) # because of the quotas in my account

<a id="test-persistent-conversation-with-rag"></a>
## Persistent Conversation Testing with AWS Bedrock Agent

This section demonstrates how to maintain context in a conversation using AWS Bedrock Agent Runtime:

- Reuses the previous session ID for continuity
- Sends a follow-up query to test conversation persistence
- Processes and displays the response, including any citations

🧠 Showcases the AI's ability to remember and reference previous interactions within the same session.

In [12]:
# we will use the same code as before, but keeping the same sessionId
query2 = 'what did you told me in the last response? also tell me why did you told me that'

model_id = "arn:aws:bedrock:us-east-1:609009159737:inference-profile/us.anthropic.claude-3-5-sonnet-20241022-v2:0"
response = ask_bedrock_llm_with_knowledge_base(query2, model_id, kb_id,sessionId=sessionId)

for i, ele in enumerate(response['citations']):
    print(f"citation {i}")
    for key, value in ele.items():
        print(f"--- key: {key}")
        print(value)

citation 0
--- key: generatedResponsePart
{'textResponsePart': {'span': {'end': 424, 'start': 0}, 'text': "While the search results don't explicitly state why NexTech Innovations chose Boston as its headquarters location, we can see that Boston plays a significant role in the company's operations. The company maintains one of its three major R&D centers there - the Boston Innovation Lab, which focuses on AI Development. Boston serves as the headquarters among other global offices located across North America, Europe, and Asia."}}
--- key: retrievedReferences
[{'content': {'text': 'Marcus Rodriguez, the company has grown from a small startup to a global technology leader.\r \r Core Products and Services:\r - AutomatePro™: Advanced robotics system for manufacturing\r - AIControl™: Machine learning platform for process optimization\r - SmartFactory™: Integrated factory management solution\r - NexSense™: IoT sensors and monitoring systems\r \r Industry Sectors:\r - Automotive Manufacturing

In [13]:
# now we will use metadata filtering

In [14]:
time.sleep(70)

In [15]:
query = "GreenLeaf"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        'text': query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn,
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': 20
                }
            }
        }
    }
)

In [16]:
for i, ele in enumerate(response['citations']):
    print(f"citation {i}")
    for key, value in ele.items():
        print(f"--- key: {key}")
        print(value)

citation 0
--- key: generatedResponsePart
{'textResponsePart': {'span': {'end': 355, 'start': 0}, 'text': 'I cannot find any information about "GreenLeaf" in the provided search results. The search results only contain information about a company called NexTech Innovations, which is a provider of artificial intelligence and robotics solutions. If you have a specific question about GreenLeaf, I would need different search results to provide an accurate answer.'}}
--- key: retrievedReferences
[{'content': {'text': "Here's a sample company profile text file:\r \r NEXTECH INNOVATIONS, INC.\r Established: 2008\r Headquarters: Boston, Massachusetts\r CEO: Dr. Sarah Chen\r Employees: 2,500+\r \r Company Overview:\r NexTech Innovations is a leading provider of artificial intelligence and robotics solutions, specializing in industrial automation and smart manufacturing technologies. Founded by Dr. Sarah Chen and Dr. Marcus Rodriguez, the company has grown from a small startup to a global techno

In [17]:
# now we will use metadata filtering to only use the file s3://virgilio-murillo-bucket/tests/bedrock/knowledge-base-tests/company-card.txt
time.sleep(70)

In [18]:
query = "nexttech"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        'text': query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn,
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'filter': {
                        'equals': {
                            'key': 'x-amz-bedrock-kb-source-uri',
                            'value': 's3://virgilio-murillo-bucket/tests/bedrock/knowledge-base-tests/company-card.txt'
                        }
                    },
                    'numberOfResults': 20
                }
            }
        }
    }
)

In [19]:
for i, ele in enumerate(response['citations']):
    print(f"citation {i}")
    for key, value in ele.items():
        print(f"--- key: {key}")
        print(value)

citation 0
--- key: generatedResponsePart
{'textResponsePart': {'span': {'end': 608, 'start': 0}, 'text': 'NexTech Innovations, Inc. is a leading provider of artificial intelligence and robotics solutions founded in 2008 and headquartered in Boston, Massachusetts. Led by CEO Dr. Sarah Chen, the company employs over 2,500 people and specializes in industrial automation and smart manufacturing technologies.\n\nThe company offers several core products including AutomatePro (robotics system), AIControl (machine learning platform), SmartFactory (factory management), and NexSense (IoT sensors). They serve multiple industries including automotive, aerospace, pharmaceutical, consumer electronics, and food processing.'}}
--- key: retrievedReferences
[{'content': {'text': 'Marcus Rodriguez, the company has grown from a small startup to a global technology leader.\r \r Core Products and Services:\r - AutomatePro™: Advanced robotics system for manufacturing\r - AIControl™: Machine learning platfor